In [16]:
# Import the OpenAI library. This library provides a Python interface to the OpenAI API.
from openai import OpenAI
import sounddevice as sd
import numpy as np
from vosk import Model, KaldiRecognizer
import json

In [17]:

# Set the sample rate and the number of channels
sample_rate = 16000
channels = 1

# Load the model
model = Model("models/vosk-model-en-us-0.22")



Exception: Failed to create a model

In [3]:
# Create an instance of the OpenAI client. The base_url parameter is set to a local server and the api_key is not needed in this case.
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

# Create a recognizer
rec = KaldiRecognizer(model, sample_rate)

# Define a global variable to keep track of the recording state
global is_recording
is_recording = True

# Define the conversation history
history = [
    {"role": "system", "content": "I'm going to give you an image idea and you are going to return me a creative and enhanced description with keywords of the image you imagine. The description should not be longer than 30 words and be specific and discriptive. Dont be afraid to be creative"},
]

# Define a callback function to process the audio chunks
def callback(indata, frames, time, status):
    global is_recording
    if rec.AcceptWaveform(indata.flatten().tobytes()):
        result = rec.Result()
        result = json.loads(result)
        if result.get('text'):  # Only print when there is text
            if "transcription pause" in result['text']:
                is_recording = False
            elif "transcription start" in result['text']:
                is_recording = True
                return  # Skip printing the "transcription start" command
            
            if is_recording:
                process_user_input(result['text'])

# Create a stream to record audio
stream = sd.InputStream(callback=callback, channels=channels, samplerate=sample_rate, dtype='int16')

def process_user_input(user_input):
    # Append user's request to history
    history.append({"role": "user", "content": user_input})

    # Generate response
    completion = client.chat.completions.create(
        model="local-model", # this field is currently unused
        messages=history,
        temperature=0.7,
    )
    # Print and append assistant's response to history
    print(completion.choices[0].message.content)
    history.append({"role": "assistant", "content": completion.choices[0].message.content})


# Start recording and processing
with stream:
    print("Recording and transcribing. Press Ctrl+C to stop.")
    while True:
        sd.sleep(200)



Recording and transcribing. Press Ctrl+C to stop.
A lush forest at sunrise, with a small cabin nestled in the trees. A couple emerging from the woods, holding hands. Keywords: forest, cabin, sunrise, couple, hand-holding.


KeyboardInterrupt: 

: 

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
The client is using an unsupported version of the Socket.IO or Engine.IO protocols (further occurrences of this error will be logged with level INFO)
127.0.0.1 - - [18/Apr/2024 12:46:55] "GET /socket.io/?EIO=3&transport=polling&t=OxmqGGc HTTP/1.1" 400 -
127.0.0.1 - - [18/Apr/2024 12:47:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2024 12:47:00] "GET /socket.io/?EIO=3&transport=polling&t=OxmqI0z HTTP/1.1" 400 -
127.0.0.1 - - [18/Apr/2024 12:47:01] "GET /socket.io/?EIO=3&transport=polling&t=OxmqI25 HTTP/1.1" 400 -
127.0.0.1 - - [18/Apr/2024 12:47:01] "GET /socket.io/?EIO=3&transport=polling&t=OxmqICd HTTP/1.1" 400 -
127.0.0.1 - - [18/Apr/2024 12:47:04] "GET /socket.io/?EIO=3&transport=polling&t=OxmqIo0 HTTP/1.1" 400 -
127.0.0.1 - - [18/Apr/2024 12:47:06] "GET /socket.io/?EIO=3&transport=polling&t=OxmqJN3 HTTP/1.1" 400 -


Microphone is ON


127.0.0.1 - - [18/Apr/2024 12:47:08] "POST /toggle_microphone HTTP/1.1" 204 -
127.0.0.1 - - [18/Apr/2024 12:47:11] "GET /socket.io/?EIO=3&transport=polling&t=OxmqKgl HTTP/1.1" 400 -
Exception ignored from cffi callback <function _StreamBase.__init__.<locals>.callback_ptr at 0x00000170B62316C0>:
Traceback (most recent call last):
  File "c:\LTAM PC\teinnStuff\SpeechToImage_Project\.venv\Lib\site-packages\sounddevice.py", line 846, in callback_ptr
    return _wrap_callback(callback, data, frames, time, status)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\LTAM PC\teinnStuff\SpeechToImage_Project\.venv\Lib\site-packages\sounddevice.py", line 2687, in _wrap_callback
    callback(*args)
  File "C:\Users\Deuka453\AppData\Local\Temp\ipykernel_16404\1408551355.py", line 33, in callback
NameError: name 'process_user_input' is not defined
127.0.0.1 - - [18/Apr/2024 12:47:16] "GET /socket.io/?EIO=3&transport=polling&t=OxmqLvI HTTP/1.1" 400 -
127.0.0.1 - - [18/Apr/2024

In [10]:
%tb

SystemExit: 1